# Foldable

_(More than just a thing as a list.)_

## The `Foldable` class

A class of data structures that can be folded to a summary value. A way of generalizing the act of folding to diﬀerent datatypes.

```haskell

class Foldable t where

  {-# MINIMAL foldMap | foldr #-}

  -- Combine the elements of a structure using a monoid.
  fold :: Monoid m => t m -> m

  -- Map each element of the structure to a monoid, and combine the results.
  foldMap :: Monoid m => (a -> m) -> t a -> m
```

## Revenge of the monoids

`fold` requires us to make the implicit `Monoid` visible in folding operations.

In [115]:
foldr (+) 0 [1..5]
fold $ map Sum [1..5]

foldr (++) "" ["hello", " julie"]
fold ["hello", " julie"]

Line 1: Use sum
Found:
foldr (+) 0
Why not:
sumLine 1: Use concat
Found:
foldr (++) ""
Why not:
concat

15

Sum {getSum = 15}

"hello julie"

"hello julie"

First argument of `foldMap` must explicitly map each element of the structure to a Monoid.

In [116]:
-- Using Monoid data constructors
foldMap Sum [1, 2, 3, 4]
foldMap Product [1, 2, 3, 4]
foldMap All [True, False, True]

-- Using a function different from the Monoid
foldMap (*5) $ map Product [1..3]
foldMap (*5) $ map Sum [1..3]

Sum {getSum = 10}

Product {getProduct = 24}

All {getAll = False}

Product {getProduct = 750}

Sum {getSum = 30}

## Demonstrating `Foldable` instances

### Identity

In [117]:
data Identity a =
    Identity a
    
instance Foldable Identity where
    foldr f z (Identity x) = f x z
    foldl f z (Identity x) = f z x
    foldMap f (Identity x) = f x
    
--

foldr (*) 1 (Identity 5)
foldl (*) 5 (Identity 5)

type PI = Product Integer
foldMap (*5) (Identity 100) :: PI

Line 1: Use product
Found:
foldr (*) 1
Why not:
product

5

25

Product {getProduct = 500}

### Maybe

In [118]:
data Optional a =
    Nada 
    | Yep a

instance Foldable Optional where
    foldr _ z Nada = z
    foldr f z (Yep x) = f x z

    foldl _ z Nada = z
    foldl f z (Yep x) = f z x

    foldMap _ Nada = mempty
    foldMap f (Yep a) = f a
    
--

- `mempty` or identity values for these Monoids
foldMap (+1) Nada :: Sum Int
foldMap (+1) Nada :: Product Int
foldMap (+1) (Just 1) :: Sum Int

Parse error (line 17, column 3): parse error on input ‘`’

## Some basic derived operations

```haskell
-- List of elements of a structure from left to right
toList :: t a -> [a]

-- Test whether the structure is empty
null :: t a -> Bool

-- Returns the size of a finite structure as an `Int`
length :: t a -> Int

-- Does the element occur in the structure?
elem :: Eq a => a -> t a -> Bool


-- The largest element of a non-empty structure
maximum :: Ord a => t a -> a

-- The least element of a non-empty structure
minimum :: Ord a => t a -> a

-- 
sum :: (Foldable t, Num a) => t a -> a

-- 
product :: (Foldable t, Num a) => t a -> a
```

In [119]:
-- Pulls value out of Maybe and into a list
toList (Just 1)

-- Pulls values out of each Maybe and into a list of lists
map toList [Just 1, Just 2, Just 3]

-- Pulls values out of each Maybe and into a list; then combines
concatMap toList [Just 1, Just 2, Just 3]

-- Nothing disappears 
concatMap toList [Just 1, Just 2, Nothing]

-- Pulls the second value into a list
toList (1, 2)

[1]

[[1],[2],[3]]

[1,2,3]

[1,2]

[2]

In [120]:
null (Left 3)
null []
null Nothing
null (1, 2)
fmap null [Just 1, Just 2, Nothing]

Line 1: Evaluate
Found:
null []
Why not:
TrueLine 1: Using null on tuple
Found:
null (1, 2)
Why not:
False

True

True

True

False

[False,False,True]

In [121]:
length (1, 2)

-- The list has 3 items
length [(1, 2), (3, 4), (5, 6)]

-- Each is 1
fmap length [(1, 2), (3, 4), (5, 6)]

-- The `a` of `Just a` in the last case above is a list; there is only one list
fmap length Just [1, 2, 3]

fmap length [Just 1, Just 2, Just 3]

fmap length [Just 1, Just 2, Nothing]

1

3

[1,1,1]

1

[1,1,1]

[1,1,0]

In [122]:
elem 2 (Just 3)
elem True (Left False)

-- Can't see Left
elem True (Left True)

elem True (Right False)
elem True (Right True)
fmap (elem 3) [Right 1, Right 2, Right 3]

Line 1: Use or
Found:
elem True
Why not:
orLine 1: Use or
Found:
elem True
Why not:
orLine 1: Use or
Found:
elem True
Why not:
orLine 1: Use or
Found:
elem True
Why not:
or

False

False

False

False

True

[False,False,True]

In [123]:
maximum [10, 12, 33, 5]
fmap maximum [Just 2, Just 10, Just 4]
fmap maximum (Just [3, 7, 10, 2])

minimum "julie"
fmap minimum (Just "julie")
fmap minimum $ map Just "jul"

33

[2,10,4]

Just 10

'e'

Just 'e'

"jul"

In [124]:
sum (7, 5)
fmap sum [(7, 5), (3, 4)]
fmap sum (Just [1, 2, 3, 4, 5])

product Nothing
fmap product (Just [])
fmap product (Right [1, 2, 3])

5

[5,4]

Just 15

1

Just 1

Right 6

## Exercises: Library Functions

Implement the functions in terms of `foldMap` or `foldr` from `Foldable`, then try them out with multiple types that have `Foldable` instances.

In [125]:
-- 1.
sum' :: (Foldable t, Num a) => t a -> a
sum' = getSum . foldMap Sum

-- 2.
product' :: (Foldable t, Num a) => t a -> a
product' = getProduct . foldMap Product

-- 3.
elem' :: (Foldable t, Eq a) => a -> t a -> Bool
elem' x = getAny . foldMap (Any . (== x))

-- 4.
minimum' :: (Foldable t, Ord a) => t a -> Maybe a
minimum' xs = Just $ foldr min acc xs
    where acc = head . toList' $ xs

-- 5.
maximum' :: (Foldable t, Ord a) => t a -> Maybe a
maximum' xs = Just $ foldr max acc xs
    where acc = head . toList' $ xs

-- 6.
null' :: (Foldable t) => t a -> Bool
null' x = not . getAny $ foldMap (const (Any True)) x

-- 7.
length' :: (Foldable t) => t a -> Int
length' = foldr f 0
    where f _ y = y + 1

-- 8.
toList' :: (Foldable t) => t a -> [a]
toList' = foldr (:) []

-- 9.
fold' :: (Foldable t, Monoid m) => t m -> m
fold' = foldMap id

-- 10.
foldMap' :: (Foldable t, Monoid m) => (a -> m) -> t a -> m
foldMap' f = foldr (mappend . f) mempty

--

xs = map Sum [1..5]

sum' xs
product' xs
elem' 2 xs
minimum' xs
maximum' xs
null' xs
length' xs
toList' xs
fold' xs
foldMap' (+1) xs

Sum {getSum = 15}

Sum {getSum = 120}

True

Just (Sum {getSum = 1})

Just (Sum {getSum = 5})

False

5

[Sum {getSum = 1},Sum {getSum = 2},Sum {getSum = 3},Sum {getSum = 4},Sum {getSum = 5}]

Sum {getSum = 15}

Sum {getSum = 20}

## Chapter Exercises

Write `Foldable` instances for the following datatypes.

In [126]:
-- 1.
data Constant a b =
    Constant b

instance Foldable (Constant a) where
  foldr _ acc _ = acc

-- 2.
data Two a b =
    Two a b

instance Foldable (Two a) where
  foldr f acc (Two a b) = f b acc

-- 3.
data Three a b c =
    Three a b c

instance Foldable (Three a b) where
  foldr f acc (Three a b c) = f c acc

-- 4.
data Three' a b =
    Three' a b b
    
instance Foldable (Three' a) where
  foldr f acc (Three' a b1 b2) = f b1 $ f b2 acc

-- 5.
data Four' a b =
    Four' a b b b

instance Foldable (Four' a) where
  foldr f acc (Four' a b1 b2 b3) = f b1 $ f b2 $ f b3 acc

In [127]:
filterF :: (Applicative f, Foldable t, Monoid (f a)) => (a -> Bool) -> t a -> f a
filterF f = foldMap g
    where
        g x = if f x then pure x
              else mempty

--

filterF odd [1, 2, 3, 4, 5] :: [Int]

[1,3,5]